In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 500)

df = pd.read_csv('k2.csv')

df.head()

,loc_rowid,pl_name,hostname,default_flag,disposition,disp_refname,sy_snum,sy_pnum,discoverymethod,disc_year,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
0,1,BD+20 594 b,BD+20 594,1,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,...,-0.01,9.37,0.02,-0.02,10.86,0.00,-0.00,2018-04-25,2017-03,2018-04-26
1,2,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,...,-0.01,9.37,0.02,-0.02,10.86,0.00,-0.00,2018-04-25,2016-10,2016-07-28
2,3,BD+20 594 b,BD+20 594,0,CONFIRMED,Espinoza et al. 2016,1,1,Transit,2016,...,-0.01,9.37,0.02,-0.02,10.86,0.00,-0.00,2018-04-25,2018-03,2018-02-15
3,4,EPIC 201111557.01,EPIC 201111557,1,CANDIDATE,Livingston et al. 2018,1,0,Transit,2018,...,-0.05,9.22,0.02,-0.02,11.40,0.00,-0.00,2018-08-02,2018-08,2018-08-02
4,5,EPIC 201111557.01,EPIC 201111557,0,CANDIDATE,Livingston et al. 2018,1,0,Transit,2018,...,-0.05,9.22,0.02,-0.02,11.40,0.00,-0.00,2018-02-15,2018-03,2018-02-15


In [55]:
df.isna().sum()

loc_rowid             0
pl_name               0
hostname              0
default_flag          0
disposition           0
disp_refname          0
sy_snum               0
sy_pnum               0
discoverymethod       0
disc_year             0
disc_facility         0
soltype               0
pl_controv_flag       0
pl_refname            0
pl_orbper            39
pl_orbpererr1       928
pl_orbpererr2       928
pl_orbperlim         39
pl_orbsmax         3062
pl_orbsmaxerr1     3069
pl_orbsmaxerr2     3069
pl_orbsmaxlim      3062
pl_rade             809
pl_radeerr1        1096
pl_radeerr2        1096
pl_radelim          809
pl_radj             809
pl_radjerr1        1096
pl_radjerr2        1096
pl_radjlim          809
pl_bmasse          3502
pl_bmasseerr1      3539
pl_bmasseerr2      3539
pl_bmasselim       3502
pl_bmassj          3502
pl_bmassjerr1      3539
pl_bmassjerr2      3539
pl_bmassjlim       3502
pl_bmassprov       3502
pl_orbeccen        3466
pl_orbeccenerr1    3655
pl_orbeccenerr2 

In [56]:
disposition_counts = df['disposition'].value_counts()
print(disposition_counts)

disposition
CONFIRMED         2156
CANDIDATE         1372
FALSE POSITIVE     293
REFUTED             18
Name: count, dtype: int64


So I think the interesting question here would be - can we build a model that can take a candidate and give a probability that it is actually a false positive. Because we don't know if candidates are or aren't, it seems like one approach we could take is to do confirmed vs refuted and false positive. If this is the case, we'll need some kind of data augmentation for these false cases or some other approach for dealing with the class imbalance.

In [57]:
df = df[df['disposition'] != 'CANDIDATE']
df['disposition'] = df['disposition'].replace({'FALSE POSITIVE': 0, 'REFUTED': 0, 'CONFIRMED': 1})

disposition_counts = df['disposition'].value_counts()
print(disposition_counts)

disposition
1    2156
0     311
Name: count, dtype: int64


C:\Users\kflin\AppData\Local\Temp\ipykernel_29616\2831736448.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['disposition'] = df['disposition'].replace({'FALSE POSITIVE': 0, 'REFUTED': 0, 'CONFIRMED': 1})


In [58]:

numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns
df_cor = df[numeric_columns]

correlations = df_cor.corr()['disposition']
correlations = correlations.drop('disposition')
correlations = correlations.abs().sort_values(ascending=False)

print("Features most correlated with target:")
pd.options.display.float_format = '{:.2f}'.format
print(correlations.head(30))

Features most correlated with target:
loc_rowid        0.61
pl_radeerr2      0.55
pl_radjerr2      0.55
pl_radeerr1      0.51
pl_radjerr1      0.51
pl_rade          0.51
pl_radj          0.51
sy_pnum          0.46
sy_dist          0.45
sy_gaiamagerr1   0.44
sy_gaiamagerr2   0.44
sy_disterr2      0.39
default_flag     0.37
sy_disterr1      0.37
st_rad           0.32
st_logg          0.28
sy_kmag          0.27
sy_kmagerr2      0.23
sy_kmagerr1      0.23
sy_gaiamag       0.22
st_met           0.22
st_masserr1      0.22
st_raderr2       0.20
st_mass          0.19
dec              0.19
st_masserr2      0.18
sy_vmag          0.16
st_raderr1       0.15
st_teff          0.14
sy_vmagerr1      0.13
Name: disposition, dtype: float64


In [ ]:
## From data_prep 

duplicates_k2 = k2_df_selected.duplicated()
print(f"Number of duplicate rows (k2): {duplicates_k2.sum()}")

duplicates_kep = kep_df_selected.duplicated()
print(f"Number of duplicate rows (kep): {duplicates_kep.sum()}")

duplicates_comb = combined_df.duplicated()
print(f"Number of duplicate rows (combined): {duplicates_comb.sum()}")